# Importing libraries
### Done by Gabriel, Colby, Heejun

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import gc

# import preprocessing classes
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# feature selection
from sklearn.feature_selection import chi2, mutual_info_regression
from sklearn.feature_selection import SelectKBest, chi2

# import models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# import train test split
from sklearn.model_selection import train_test_split

# feature importance
from sklearn.inspection import permutation_importance

# import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

# functions to save data
from joblib import dump, load

# read data
## mount google drive
from google.colab import drive
drive.mount("/content/drive/")
%cd '/content/drive/MyDrive/DS440/dataset/Data/'
!pwd

## read in files
dir = '/content/drive/MyDrive/DS440/dataset/Data/'
cnx = sqlite3.connect(dir + 'wildfire.sqlite')
cursor = cnx.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='Fires';")
tables = cursor.fetchall()
Fires = pd.read_sql_query("SELECT * FROM 'Fires'", cnx).set_index('FOD_ID')
gc.collect()
# Drop unnecessary column
Fires.drop(columns=['FPA_ID','SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM','NWCG_REPORTING_AGENCY','NWCG_REPORTING_UNIT_ID',
                    'NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID',
                    'LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_PLUS_INCIDENT_JOIN_ID','ICS_209_PLUS_COMPLEX_JOIN_ID',
                    'MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME','OWNER_DESCR','NWCG_CAUSE_CLASSIFICATION',
                    'NWCG_CAUSE_AGE_CATEGORY','FIRE_SIZE_CLASS','CONT_TIME','CONT_DATE'],inplace=True)

Mounted at /content/drive/
/content/drive/MyDrive/DS440/dataset/Data
/content/drive/MyDrive/DS440/dataset/Data


In [2]:
Fires.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
1,2005,2/2/2005 0:00,33,1300.0,Power generation/transmission/distribution,33.0,0.10,40.036944,-121.005833,CA,63,06063,Plumas County
2,2004,5/12/2004 0:00,133,845.0,Natural,133.0,0.25,38.933056,-120.404444,CA,61,06061,Placer County
3,2004,5/31/2004 0:00,152,1921.0,Debris and open burning,152.0,0.10,38.984167,-120.735556,CA,17,06017,El Dorado County
4,2004,6/28/2004 0:00,180,1600.0,Natural,185.0,0.10,38.559167,-119.913333,CA,3,06003,Alpine County
5,2004,6/28/2004 0:00,180,1600.0,Natural,185.0,0.10,38.559167,-119.933056,CA,3,06003,Alpine County


In [4]:
#ToCheck = Fires.drop(columns=['FIRE_YEAR','DISCOVERY_DATE','DISCOVERY_DOY','NWCG_GENERAL_CAUSE','CONT_DOY','LATITUDE','LONGITUDE','STATE','COUNTY','FIPS_CODE','FIPS_NAME'])
#NaN = ToCheck[ToCheck['DISCOVERY_TIME'].isna()]
NaN = Fires[Fires['DISCOVERY_TIME'].isna()]
NaN

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
174655,1993,6/17/1993 0:00,168,NaN,Natural,168.0,0.30,66.583000,-145.252600,AK,None,None,None
175122,1993,6/1/1993 0:00,152,NaN,Natural,152.0,0.10,62.032600,-163.485700,AK,None,None,None
180687,1994,5/26/1994 0:00,146,NaN,Missing data/not specified/undetermined,146.0,1.00,44.073600,-99.440400,SD,None,None,None
190717,1994,7/4/1994 0:00,185,NaN,Fireworks,185.0,0.10,44.749800,-121.251200,OR,None,None,None
196902,1994,8/5/1994 0:00,217,NaN,Missing data/not specified/undetermined,217.0,0.10,48.483300,-108.767400,MT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400431685,2010,4/10/2010 0:00,100,NaN,Missing data/not specified/undetermined,NaN,60.40,43.036149,-97.441521,SD,Yankton,46135,Yankton County
400431687,2010,7/18/2010 0:00,199,NaN,Natural,NaN,1.30,44.099200,-103.317500,SD,PENNINGTON,46103,Pennington County
400431688,2010,7/20/2010 0:00,201,NaN,Missing data/not specified/undetermined,NaN,0.25,44.617264,-103.385940,SD,Butte,46019,Butte County


In [5]:
a = Fires.sort_values('FIRE_SIZE')
NaN_a = a[a['DISCOVERY_TIME'].isna()]
NaN_a

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
201816381,2013,12/18/2013 0:00,352,NaN,Missing data/not specified/undetermined,NaN,0.00001,44.997000,-101.233300,SD,Dewey,46041,Dewey County
850766,1999,10/30/1999 0:00,303,NaN,Missing data/not specified/undetermined,NaN,0.00010,43.536440,-96.860460,SD,None,None,None
850630,1999,11/2/1999 0:00,306,NaN,Equipment and vehicle use,NaN,0.00010,44.389590,-99.471440,SD,None,None,None
850801,1999,12/29/1999 0:00,363,NaN,Missing data/not specified/undetermined,NaN,0.00010,43.507650,-96.681020,SD,None,None,None
400116141,2017,10/16/2017 0:00,289,NaN,Equipment and vehicle use,NaN,0.00010,43.875600,-99.737400,SD,Lyman,46085,Lyman County
...,...,...,...,...,...,...,...,...,...,...,...,...,...
403500,2002,5/26/2002 0:00,146,NaN,Natural,248.0,84015.00000,63.618700,-155.820600,AK,None,None,None
553116,2008,8/22/2008 0:00,235,NaN,Natural,NaN,102379.53000,46.237700,-108.341700,MT,Yellowstone,30111,Yellowstone County
201752827,2011,4/14/2011 0:00,104,NaN,Missing data/not specified/undetermined,NaN,106027.00000,32.237396,-102.917336,TX,None,None,None


# Data Preprocessing
## convert all features into usable format
## Use random sample from originial dataset due to high computational cost
## Take 1% of original data
## Purpose : 
1. To handle big data
2. To debug and to see that the code actually works

In [6]:
# Later just remove this line of code when using original dataset
Fires = Fires.sample(frac=0.01,replace=False)

In [7]:
Fires

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE,CONT_DOY,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
FOD_ID,,,,,,,,,,,,,
1057363,2002,2/1/2002 0:00,32,NaN,Debris and open burning,NaN,12.0,33.178900,-79.940800,SC,None,None,None
962296,1998,5/18/1998 0:00,138,NaN,Missing data/not specified/undetermined,NaN,0.3,33.761400,-87.747500,AL,None,None,None
1796575,2001,4/22/2001 0:00,112,1630.0,Equipment and vehicle use,112.0,0.2,37.578516,-80.863110,WV,Summers,54089,Summers County
400305889,2018,7/23/2018 0:00,204,1144.0,Recreation and ceremony,271.0,560.4,64.958600,-143.527500,AK,Dillingham,02070,Dillingham Census Area
195785,1992,4/3/1992 0:00,94,1935.0,Misuse of fire by a minor,94.0,5.0,45.516600,-107.617300,MT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131933,2000,8/22/2000 0:00,235,1715.0,Natural,235.0,0.1,38.775000,-112.400556,UT,27,49027,Millard County
1465839,2010,4/14/2010 0:00,104,NaN,Arson/incendiarism,NaN,0.4,45.026833,-67.180667,ME,Washington,23029,Washington County
897991,1992,9/28/1992 0:00,272,1505.0,Debris and open burning,272.0,0.3,45.843794,-92.705024,WI,Burnett,55013,Burnett County


# Colby's Coding

In [8]:
# extract month from discovery date
def get_first_element(date_list):
  return date_list[0]

Fires['DISCOVERY_DATE'] = Fires['DISCOVERY_DATE'].astype("string")
Fires['DISCOVERY_LIST'] = Fires['DISCOVERY_DATE'].str.split(pat='/')
Fires['DISCOVERY_MONTH'] = Fires['DISCOVERY_LIST'].map(get_first_element)
Fires['DISCOVERY_MONTH'] = Fires['DISCOVERY_MONTH'].astype("int64")

# encode causes into integers
Fires['NWCG_GENERAL_CAUSE'] = Fires['NWCG_GENERAL_CAUSE'].astype("string")
cause_enc = LabelEncoder()
Fires['NWCG_CAUSE_CLASSIFICATION_ORD'] = cause_enc.fit_transform(Fires['NWCG_GENERAL_CAUSE'])


# figure out number of days to contain the fire
Fires['DAYS_TO_CONT'] = Fires['CONT_DOY'] - Fires['DISCOVERY_DOY']
##### if it was contained the next year, need to do something about that
def convert_negative_days(day):
  if day < 0:
    return 365 + day
  else:
    return day

Fires['DAYS_TO_CONT'] = Fires['DAYS_TO_CONT'].map(convert_negative_days)


# encode state categories into integers
Fires['STATE'] = Fires['STATE'].astype("string")
state_enc = LabelEncoder()
Fires['STATE_ORD'] = state_enc.fit_transform(Fires['STATE'])

# what to do about counties? Use fips code (must be an int)
Fires['FIPS_CODE'] = Fires['FIPS_CODE'].astype("string")
Fires['FIPS_CODE'].fillna('1000000',inplace=True)
Fires['FIPS_CODE'] = Fires['FIPS_CODE'].astype("int64")

# fill NA with mean of day to containment (mean = 0.92)
Fires['DAYS_TO_CONT'].fillna(value=Fires['DAYS_TO_CONT'].mean(),inplace=True)


# drop date and list
Fires.drop(columns=['DISCOVERY_DATE','DISCOVERY_LIST','NWCG_GENERAL_CAUSE',
                    'CONT_DOY','DISCOVERY_DOY','STATE','FIPS_NAME',
                    'COUNTY'],inplace=True)
gc.collect()

30

In [9]:
Fires.head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
1057363,2002,NaN,12.0,33.178900,-79.94080,1000000,2,1,0.999622,40
962296,1998,NaN,0.3,33.761400,-87.74750,1000000,5,5,0.999622,1
1796575,2001,1630.0,0.2,37.578516,-80.86311,54089,4,2,0.000000,49
400305889,2018,1144.0,560.4,64.958600,-143.52750,2070,7,11,67.000000,0
195785,1992,1935.0,5.0,45.516600,-107.61730,1000000,4,6,0.000000,25


#** Code by Heejun Son**
### Need to handle missing values in DISCOVERY_TIME column 
### Need to compare MICE vs KNN vs Replacement with the most frequent number
I did not drop DISCOVERY_TIME column.


In [10]:
# To see whether there is missing values
print("No. NaN in FIRE_YEAR: ", Fires['FIRE_YEAR'].isna().sum())
print("No. NaN in DISCOVERY_TIME: ", Fires['DISCOVERY_TIME'].isna().sum())
print("No. NaN in FIRE_SIZE: ", Fires['FIRE_SIZE'].isna().sum())
print("No. NaN in LATITUDE: ", Fires['LATITUDE'].isna().sum())
print("No. NaN in LONGITUDE: ", Fires['LONGITUDE'].isna().sum())
print("No. NaN in FIPS_CODE: ", Fires['FIPS_CODE'].isna().sum())
print("No. NaN in DISCOVERY_MONTH: ", Fires['DISCOVERY_MONTH'].isna().sum())
print("No. NaN in NWCG_CAUSE_CLASSIFICATION_ORD: ", Fires['NWCG_CAUSE_CLASSIFICATION_ORD'].isna().sum())
print("No. NaN in DAYS_TO_CONT: ", Fires['DAYS_TO_CONT'].isna().sum())
print("No. NaN in STATE_ORD: ", Fires['STATE_ORD'].isna().sum())

No. NaN in FIRE_YEAR:  0
No. NaN in DISCOVERY_TIME:  7405
No. NaN in FIRE_SIZE:  0
No. NaN in LATITUDE:  0
No. NaN in LONGITUDE:  0
No. NaN in FIPS_CODE:  0
No. NaN in DISCOVERY_MONTH:  0
No. NaN in NWCG_CAUSE_CLASSIFICATION_ORD:  0
No. NaN in DAYS_TO_CONT:  0
No. NaN in STATE_ORD:  0


In [11]:
#Drop all the NaN values
Fires_Drop = Fires.sample(frac=1,random_state=0).dropna()
Fires_Drop

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
160003,2003,917.0,0.20,43.826389,-110.947778,56039,8,7,0.0,50
146688,2001,900.0,0.10,33.635000,-109.468333,4011,6,7,0.0,3
400361649,2018,1643.0,1.00,42.308250,-86.250150,26159,3,5,0.0,21
136905,2001,1550.0,10.00,33.766667,-110.563889,1000000,6,7,1.0,3
400088134,2016,2335.0,0.25,39.520800,-74.806700,34001,4,8,10.0,30
...,...,...,...,...,...,...,...,...,...,...
201639643,2012,1441.0,0.78,33.021264,-83.238747,13009,10,6,0.0,9
400057472,2016,1131.0,0.25,42.060570,-71.022280,25023,4,1,0.0,18
300220691,2015,1728.0,50.00,33.768883,-103.378473,35041,6,7,0.0,31


In [12]:
#Replace with the most_Frequent value
Fires_Freq = Fires.sample(frac=1,random_state=0)
Fires_Freq['DISCOVERY_TIME'] = Fires_Freq[['DISCOVERY_TIME']].fillna(Fires_Freq[['DISCOVERY_TIME']].mode().iloc[0])
Fires_Freq

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD
FOD_ID,,,,,,,,,,
160003,2003,917.0,0.20,43.826389,-110.947778,56039,8,7,0.000000,50
146688,2001,900.0,0.10,33.635000,-109.468333,4011,6,7,0.000000,3
400361649,2018,1643.0,1.00,42.308250,-86.250150,26159,3,5,0.000000,21
201415626,2011,1400.0,0.25,33.034865,-97.446765,48497,7,5,0.999622,43
400401028,2001,1400.0,0.10,35.746200,-92.073290,5137,3,5,0.999622,2
...,...,...,...,...,...,...,...,...,...,...
300220691,2015,1728.0,50.00,33.768883,-103.378473,35041,6,7,0.000000,31
1010801,1997,1400.0,0.10,30.796400,-92.441100,1000000,10,5,0.999622,17
485406,2007,1400.0,0.05,31.347977,-83.002595,13003,2,1,0.999622,9


In [13]:
#Replace NaN with values predicted by KNN
from sklearn.impute import KNNImputer
Fires_KNN = Fires.sample(frac=1,random_state=0)
Input = Fires_KNN[['FIRE_YEAR','DISCOVERY_TIME','FIRE_SIZE','LATITUDE','LONGITUDE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD']]
imputer = KNNImputer(n_neighbors=5)

In [14]:
result = imputer.fit_transform(Input)
check = pd.DataFrame(result)
New_Discovery_Time = check[[1]].values.tolist()

In [15]:
Fires_KNN['Assumption'] = New_Discovery_Time
Fires_KNN.head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD,Assumption
FOD_ID,,,,,,,,,,,
160003,2003,917.0,0.20,43.826389,-110.947778,56039,8,7,0.000000,50,[917.0]
146688,2001,900.0,0.10,33.635000,-109.468333,4011,6,7,0.000000,3,[900.0]
400361649,2018,1643.0,1.00,42.308250,-86.250150,26159,3,5,0.000000,21,[1643.0]
201415626,2011,NaN,0.25,33.034865,-97.446765,48497,7,5,0.999622,43,[1242.6]
400401028,2001,NaN,0.10,35.746200,-92.073290,5137,3,5,0.999622,2,[1740.8]


In [16]:
# Convert object to float64
Fires_KNN['Assumption'] = Fires_KNN['Assumption'].astype("string")
Fires_KNN['Assumption'] = Fires_KNN['Assumption'].str.strip('[]')
Fires_KNN['Assumption'] = Fires_KNN['Assumption'].astype("float64")
# Drop DISCOVERY_TIME column that contains NaN
Fires_KNN.drop(columns=['DISCOVERY_TIME'],inplace=True)
# Change column name from Assumption to DISCOVERY_TIME
Fires_KNN['DISCOVERY_TIME'] = Fires_KNN['Assumption']
Fires_KNN.drop(columns=['Assumption'],inplace=True)
Fires_KNN

,FIRE_YEAR,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD,DISCOVERY_TIME
FOD_ID,,,,,,,,,,
160003,2003,0.20,43.826389,-110.947778,56039,8,7,0.000000,50,917.0
146688,2001,0.10,33.635000,-109.468333,4011,6,7,0.000000,3,900.0
400361649,2018,1.00,42.308250,-86.250150,26159,3,5,0.000000,21,1643.0
201415626,2011,0.25,33.034865,-97.446765,48497,7,5,0.999622,43,1242.6
400401028,2001,0.10,35.746200,-92.073290,5137,3,5,0.999622,2,1740.8
...,...,...,...,...,...,...,...,...,...,...
300220691,2015,50.00,33.768883,-103.378473,35041,6,7,0.000000,31,1728.0
1010801,1997,0.10,30.796400,-92.441100,1000000,10,5,0.999622,17,1561.2
485406,2007,0.05,31.347977,-83.002595,13003,2,1,0.999622,9,1502.4


In [17]:
#Replace NaN with values predicted by Multivariate feature imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
Fires_MICE = Fires.sample(frac=1,random_state=0)
Input = Fires_KNN[['FIRE_YEAR','DISCOVERY_TIME','FIRE_SIZE','LATITUDE','LONGITUDE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD']]
imp_15 = IterativeImputer(max_iter=15, random_state=0)
imp_15.fit(Input)
result_mice = imp_15.transform(Input)
check_mice = pd.DataFrame(result_mice)
New_Discovery_Time_mice = check_mice[[1]].values.tolist()

In [18]:
Fires_MICE['Assumption'] = New_Discovery_Time_mice
Fires_MICE.head()

,FIRE_YEAR,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD,Assumption
FOD_ID,,,,,,,,,,,
160003,2003,917.0,0.20,43.826389,-110.947778,56039,8,7,0.000000,50,[917.0]
146688,2001,900.0,0.10,33.635000,-109.468333,4011,6,7,0.000000,3,[900.0]
400361649,2018,1643.0,1.00,42.308250,-86.250150,26159,3,5,0.000000,21,[1643.0]
201415626,2011,NaN,0.25,33.034865,-97.446765,48497,7,5,0.999622,43,[1242.6]
400401028,2001,NaN,0.10,35.746200,-92.073290,5137,3,5,0.999622,2,[1740.8]


In [19]:
Fires_MICE.dtypes

FIRE_YEAR                          int64
DISCOVERY_TIME                   float64
FIRE_SIZE                        float64
LATITUDE                         float64
LONGITUDE                        float64
FIPS_CODE                          int64
DISCOVERY_MONTH                    int64
NWCG_CAUSE_CLASSIFICATION_ORD      int64
DAYS_TO_CONT                     float64
STATE_ORD                          int64
Assumption                        object
dtype: object

In [20]:
# Convert object to float64
Fires_MICE['Assumption'] = Fires_MICE['Assumption'].astype("string")
Fires_MICE['Assumption'] = Fires_MICE['Assumption'].str.strip('[]')
Fires_MICE['Assumption'] = Fires_MICE['Assumption'].astype("float64")
# Drop DISCOVERY_TIME column that contains NaN
Fires_MICE.drop(columns=['DISCOVERY_TIME'],inplace=True)
# Change column name from Assumption to DISCOVERY_TIME
Fires_MICE['DISCOVERY_TIME'] = Fires_MICE['Assumption']
Fires_MICE.drop(columns=['Assumption'],inplace=True)
Fires_MICE

,FIRE_YEAR,FIRE_SIZE,LATITUDE,LONGITUDE,FIPS_CODE,DISCOVERY_MONTH,NWCG_CAUSE_CLASSIFICATION_ORD,DAYS_TO_CONT,STATE_ORD,DISCOVERY_TIME
FOD_ID,,,,,,,,,,
160003,2003,0.20,43.826389,-110.947778,56039,8,7,0.000000,50,917.0
146688,2001,0.10,33.635000,-109.468333,4011,6,7,0.000000,3,900.0
400361649,2018,1.00,42.308250,-86.250150,26159,3,5,0.000000,21,1643.0
201415626,2011,0.25,33.034865,-97.446765,48497,7,5,0.999622,43,1242.6
400401028,2001,0.10,35.746200,-92.073290,5137,3,5,0.999622,2,1740.8
...,...,...,...,...,...,...,...,...,...,...
300220691,2015,50.00,33.768883,-103.378473,35041,6,7,0.000000,31,1728.0
1010801,1997,0.10,30.796400,-92.441100,1000000,10,5,0.999622,17,1561.2
485406,2007,0.05,31.347977,-83.002595,13003,2,1,0.999622,9,1502.4


# Create a baseline mse and mae

In [21]:
FIRE_SIZE_PERMUTE = Fires['FIRE_SIZE'].sample(frac=1,random_state=0)

In [22]:
print("MAE: ",mean_absolute_error(FIRE_SIZE_PERMUTE,Fires['FIRE_SIZE']))
print("MSE: ",mean_squared_error(FIRE_SIZE_PERMUTE,Fires['FIRE_SIZE']))

MAE:  178.58154345717185
MSE:  25564017.087541316


# ** Code by Heejun **
## Feature selection information
Later need to convert LONGITUDE to negative values!!

Reason for using abs() : There are no positive values !!
                        
(United States only have negative lonitude values)

In [23]:
# Convert negative value to positive
# Feature Selection must have non-negative values
Fires_Drop['LONGITUDE'] = Fires_Drop['LONGITUDE'].abs()
Fires_Freq['LONGITUDE'] = Fires_Freq['LONGITUDE'].abs()
Fires_KNN['LONGITUDE'] = Fires_KNN['LONGITUDE'].abs()
Fires_MICE['LONGITUDE'] = Fires_MICE['LONGITUDE'].abs()

In [24]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
############################################################################################################################################################
X_Drop = Fires_Drop[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_Drop = Fires_Drop['FIRE_SIZE']
# Transform FIRE_SIZE
# SelectKBest() cannot handle countinous numbers
Encoder_Drop = preprocessing.LabelEncoder()
Firs_size_encoded_Drop = Encoder_Drop.fit_transform(y_Drop)

Best_Features_Drop = dict()
for i in range(1,9):
  selector_Drop = SelectKBest(chi2, k=i)
  selector_Drop.fit(X_Drop, Firs_size_encoded_Drop)
  Features_Drop = list(X_Drop.columns[selector_Drop.get_support(indices=True)])
  Best_Features_Drop[i] = Features_Drop
  print("(Drop) When k= ", i, ",", Features_Drop)

############################################################################################################################################################
X_Freq = Fires_Freq[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_Freq = Fires_Freq['FIRE_SIZE']
# Transform FIRE_SIZE
# SelectKBest() cannot handle countinous numbers
Encoder_Freq = preprocessing.LabelEncoder()
Firs_size_encoded_Freq = Encoder_Freq.fit_transform(y_Freq)

Best_Features_Freq = dict()
for i in range(1,9):
  selector_Freq = SelectKBest(chi2, k=i)
  selector_Freq.fit(X_Freq, Firs_size_encoded_Freq)
  Features_Freq = list(X_Freq.columns[selector_Freq.get_support(indices=True)])
  Best_Features_Freq[i] = Features_Freq
  print("(Freq) When k= ", i, ",", Features_Freq)

############################################################################################################################################################
X_KNN = Fires_KNN[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_KNN = Fires_KNN['FIRE_SIZE']
# Transform FIRE_SIZE
# SelectKBest() cannot handle countinous numbers
Encoder_KNN = preprocessing.LabelEncoder()
Firs_size_encoded_KNN = Encoder_KNN.fit_transform(y_KNN)

Best_Features_KNN = dict()
for i in range(1,9):
  selector_KNN = SelectKBest(chi2, k=i)
  selector_KNN.fit(X_KNN, Firs_size_encoded_KNN)
  Features_KNN = list(X_KNN.columns[selector_KNN.get_support(indices=True)])
  Best_Features_KNN[i] = Features_KNN
  print("(KNN) When k= ", i, ",", Features_KNN)
  ############################################################################################################################################################
X_MICE = Fires_MICE[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_MICE = Fires_MICE['FIRE_SIZE']
# Transform FIRE_SIZE
# SelectKBest() cannot handle countinous numbers
Encoder_MICE = preprocessing.LabelEncoder()
Firs_size_encoded_MICE = Encoder_MICE.fit_transform(y_MICE)

Best_Features_MICE = dict()
for i in range(1,9):
  selector_MICE = SelectKBest(chi2, k=i)
  selector_MICE.fit(X_MICE, Firs_size_encoded_MICE)
  Features_MICE = list(X_MICE.columns[selector_MICE.get_support(indices=True)])
  Best_Features_MICE[i] = Features_MICE
  print("(MICE) When k= ", i, ",", Features_MICE)

(Drop) When k=  1 , ['FIPS_CODE']
(Drop) When k=  2 , ['FIPS_CODE', 'DAYS_TO_CONT']
(Drop) When k=  3 , ['FIPS_CODE', 'DAYS_TO_CONT', 'DISCOVERY_TIME']
(Drop) When k=  4 , ['FIPS_CODE', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Drop) When k=  5 , ['LONGITUDE', 'FIPS_CODE', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Drop) When k=  6 , ['LONGITUDE', 'FIPS_CODE', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Drop) When k=  7 , ['LATITUDE', 'LONGITUDE', 'FIPS_CODE', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Drop) When k=  8 , ['LATITUDE', 'LONGITUDE', 'FIPS_CODE', 'DISCOVERY_MONTH', 'NWCG_CAUSE_CLASSIFICATION_ORD', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Freq) When k=  1 , ['FIPS_CODE']
(Freq) When k=  2 , ['FIPS_CODE', 'DAYS_TO_CONT']
(Freq) When k=  3 , ['FIPS_CODE', 'DAYS_TO_CONT', 'DISCOVERY_TIME']
(Freq) When k=  4 , ['FIPS_CODE', 'DAYS_TO_CONT', 'STATE_ORD', 'DISCOVERY_TIME']
(Freq) When k=  

Freatures to predict class:
- fire year, general cause, lat, long

Features to predict cause:
- fire year, size class, lat, long

# MODEL

In [25]:
#### split data into training, validation, and testing
X_Drop = Fires_Drop[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_Drop = Fires_Drop['FIRE_SIZE']
X_Drop_train, X_Drop_test, y_Drop_train, y_Drop_test = train_test_split(X_Drop, y_Drop, test_size=0.2, random_state=0)
######################################################################################################################################################################
X_Freq = Fires_Freq[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_Freq = Fires_Freq['FIRE_SIZE']
X_Freq_train, X_Freq_test, y_Freq_train, y_Freq_test = train_test_split(X_Freq, y_Freq, test_size=0.2, random_state=0)
######################################################################################################################################################################
X_KNN = Fires_KNN[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_KNN = Fires_KNN['FIRE_SIZE']
X_KNN_train, X_KNN_test, y_KNN_train, y_KNN_test = train_test_split(X_KNN, y_KNN, test_size=0.2, random_state=0)
######################################################################################################################################################################
X_MICE = Fires_MICE[['FIRE_YEAR','LATITUDE','LONGITUDE','FIPS_CODE','DISCOVERY_MONTH','NWCG_CAUSE_CLASSIFICATION_ORD','DAYS_TO_CONT','STATE_ORD','DISCOVERY_TIME']]
y_MICE = Fires_MICE['FIRE_SIZE']
X_MICE_train, X_MICE_test, y_MICE_train, y_MICE_test = train_test_split(X_MICE, y_MICE, test_size=0.2, random_state=0)

In [26]:
#### list of models to train
# xgboost, linear regression (with and without regularization), random forest, naive bayes, nearest neighbors
# k-means, svm
model_list = [LinearRegression, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor,
               KNeighborsRegressor, SVR] #GaussianNB, 
################################################################################################################################               
model_scores_Drop = dict()
for model in model_list:
  # fit model to data
  print(model)
  regressor_Drop = model().fit(X_Drop_train,y_Drop_train)
  y_pred_Drop = regressor_Drop.predict(X_Drop_test)
  model_Drop_mse = mean_squared_error(y_pred_Drop,y_Drop_test)
  model_Drop_mae = mean_absolute_error(y_pred_Drop,y_Drop_test)
  model_scores_Drop[model] = (regressor_Drop,y_pred_Drop,model_Drop_mse,model_Drop_mae)
  print("(Drop) MSE = ", model_Drop_mse, '\n', "MAE = ", model_Drop_mae)
################################################################################################################################               
model_scores_Freq = dict()
for model in model_list:
  # fit model to data
  print(model)
  regressor_Freq = model().fit(X_Freq_train,y_Freq_train)
  y_Freq_pred = regressor_Freq.predict(X_Freq_test)
  model_Freq_mse = mean_squared_error(y_Freq_pred,y_Freq_test)
  model_Freq_mae = mean_absolute_error(y_Freq_pred,y_Freq_test)
  model_scores_Freq[model] = (regressor_Freq,y_Freq_pred,model_Freq_mse,model_Freq_mae)
  print("(Freq) MSE = ", model_Freq_mse, '\n', "(Freq) MAE = ", model_Freq_mae)
################################################################################################################################
model_scores_KNN = dict()
for model in model_list:
  # fit model to data
  print(model)
  regressor_KNN = model().fit(X_KNN_train,y_KNN_train)
  y_KNN_pred = regressor_KNN.predict(X_KNN_test)
  model_KNN_mse = mean_squared_error(y_KNN_pred,y_KNN_test)
  model_KNN_mae = mean_absolute_error(y_KNN_pred,y_KNN_test)
  model_scores_KNN[model] = (regressor_KNN,y_KNN_pred,model_KNN_mse,model_KNN_mae)
  print("(KNN) MSE = ", model_KNN_mse, '\n', "(KNN) MAE = ", model_KNN_mae)
################################################################################################################################
model_scores_MICE = dict()
for model in model_list:
  # fit model to data
  print(model)
  regressor_MICE = model().fit(X_MICE_train,y_MICE_train)
  y_MICE_pred = regressor_MICE.predict(X_MICE_test)
  model_MICE_mse = mean_squared_error(y_MICE_pred,y_MICE_test)
  model_MICE_mae = mean_absolute_error(y_MICE_pred,y_MICE_test)
  model_scores_MICE[model] = (regressor_MICE,y_MICE_pred,model_MICE_mse,model_MICE_mae)
  print("(MICE) MSE = ", model_MICE_mse, '\n', "(MICE) MAE = ", model_MICE_mae)  

<class 'sklearn.linear_model._base.LinearRegression'>
(Drop) MSE =  10579733.408207225 
 MAE =  329.2778079014029
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
(Drop) MSE =  15084597.886855533 
 MAE =  268.0045854725435
<class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
(Drop) MSE =  12727665.554909015 
 MAE =  196.8532773562
<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>
(Drop) MSE =  12063481.473030018 
 MAE =  735.3813411670889
<class 'sklearn.neighbors._regression.KNeighborsRegressor'>
(Drop) MSE =  11126719.702084161 
 MAE =  171.42317240273397
<class 'sklearn.svm._classes.SVR'>
(Drop) MSE =  11533937.373846335 
 MAE =  136.24323428716679
<class 'sklearn.linear_model._base.LinearRegression'>
(Freq) MSE =  2106582.9294672427 
 (Freq) MAE =  204.051211848415
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
(Freq) MSE =  3658960.8906637277 
 (Freq) MAE =  131.9239451893434
<class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
(Freq) MSE =  4

# Save Models

In [ ]:
from joblib import dump, load

In [ ]:
for m in model_scores.keys():
  # save model object
  dump(model_scores[m][0],'/content/drive/MyDrive/DS 440/Models/ModelIs%s.joblib' %str(model_scores[m][0]))
  # save predictions
  dump(model_scores[m][1],'/content/drive/MyDrive/DS 440/Predictions/PredFor%s.joblib' %str(model_scores[m][0]))
  

In [ ]:
# visualize features - might need more notebooks for this




In [ ]:
# develop model using hyperopt



In [ ]:
# use permutation importance on each feature



In [ ]:
 # put results in a dataframe

 

In [ ]:
############### SAVE FOR LATER ###########################
# polynomial features


# determine which ones are useful 
## use mutual information criterion to do this

